# Data Loading

**New variables:**

Demand:
- [x] Actual values and generation mix 
    (https://www.neso.energy/data-portal/historic-demand-data)
- [x] Day-ahead forecasts 
    (https://www.neso.energy/data-portal/1-day-ahead-demand-forecast/historic_day_ahead_demand_forecasts)

Generation:
- [x] Historic generation mix 
    (https://www.neso.energy/data-portal/historic-generation-mix)
- [x] Day-ahead metered wind forecasts
    (https://www.neso.energy/data-portal/day-ahead-wind-forecast)
- [x] Embedded wind and solar forecasts 
    (https://www.neso.energy/data-portal/embedded-wind-and-solar-forecasts)

Prices:
- [x] Electricity prices (BMRS MIP)
- [x] Natural gas prices (ONS, already sourced)
- [ ] Coal prices
- [ ] Carbon credit prices

*Note: Prices of futures contracts would be preferable, 
but these are often behind paywalls.*

Temporal features:
- [x] `holidays` package

(Optional) Capacity:
- Nuclear capacity
- Gas capacity
- Interconnector capacity

In [ ]:
import os
from datetime import datetime, timedelta
import requests
import numpy as np
import pandas as pd

## Historical Actual Demand Data

In [ ]:
start_date = "2020-12-31"
end_date = "2025-01-02"

In [ ]:
url = "https://api.neso.energy/api/3/action/datastore_search_sql"
resource_id = "b2bde559-3455-4021-b179-dfe60c0337b0"

query = f"""
SELECT *
FROM "{resource_id}"
WHERE "SETTLEMENT_DATE" >= '{start_date}' AND "SETTLEMENT_DATE" <= '{end_date}'
ORDER BY "SETTLEMENT_DATE" ASC
"""

params = {"sql": query}

r = requests.get(url, params=params)
if r.status_code == 200:
    demand_actual = r.json()
else:
    raise Exception("Request Failed")

In [ ]:
# Unpack data into a dataframe
# note: ND = national demand, TSD = transition system demand
df_demand_actual = pd.DataFrame(demand_actual['result']['records'])
df_demand_actual.columns

In [ ]:
df_demand_actual['SETTLEMENT_DATE'] = pd.to_datetime(df_demand_actual['SETTLEMENT_DATE'])

# keep only required columns
df_demand_actual = df_demand_actual[['ND', 'TSD', 'SETTLEMENT_DATE', 'SETTLEMENT_PERIOD']]


df_demand_actual.head()

## Historical Generation Mix

In [ ]:
url = "https://api.neso.energy/api/3/action/datastore_search_sql"
resource_id = "f93d1835-75bc-43e5-84ad-12472b180a98"

query = f"""
SELECT *
FROM "{resource_id}"
WHERE "DATETIME" >= '{start_date}' AND "DATETIME" <= '{end_date}'
ORDER BY "DATETIME" ASC
"""

params = {"sql": query}

r = requests.get(url, params=params)
if r.status_code == 200:
    generation_actual = r.json()
else:
    raise Exception("Request Failed")

In [ ]:
# Unpack data into a dataframe
df_generation_actual = pd.DataFrame(generation_actual['result']['records'])
df_generation_actual.columns

In [ ]:
# set timestamp as index
df_generation_actual['DATETIME'] = pd.to_datetime(df_generation_actual['DATETIME'])
df_generation_actual.set_index('DATETIME', inplace=True)

# drop unnecessary columns
df_generation_actual.drop(columns=['_id', '_full_text', 'FOSSIL', 'RENEWABLE', 'ZERO_CARBON', 'LOW_CARBON', 'CARBON_INTENSITY'], inplace=True)
df_generation_actual.drop(columns=list(df_generation_actual.filter(regex='_perc')), inplace=True)

# fix non-numeric columns
df_generation_actual = df_generation_actual.apply(pd.to_numeric, errors='coerce')

# check data is correct
calculated_generation = df_generation_actual.drop(columns='GENERATION').sum(axis=1)
relative_error = (df_generation_actual['GENERATION'] - calculated_generation.abs()) / calculated_generation.abs()
assert np.allclose(relative_error, 0, atol=1e-3)

df_generation_actual.head()

## Historical Day-Ahead Demand Forecasts

*Definitions:*

Cardinal Point: Electricity demand fluctuates during a day depending on 
how much energy people, businesses and industries are using at that 
moment in time. As this electricity demand goes up and down we get 
characteristic peaks and troughs, with some of these peaks and troughs 
appearing every single day at similar times. These we call cardinal 
points and are the points during the day that we forecast demand for.

Cardinal Point Type: Fixed, Trough or Peak. Cardinal points (CPs) can 
either be fixed (occur at a fixed time), trough (minimum demands during 
a set period of the day) or peak(maximum demands during a set period of 
the day). These are represented throught the first letter of the point 
type (F,T or P)

Cardinal Point Start Time: The time when a particular cardinal point 
(CP) starts during the day. This is given relative to the timezone in 
effect in the UK at the forecast time and date.

Cardinal Point End Time: The time when a particular cardinal point (CP) 
ends during the day. This is given relative to the timezone in effect 
in the UK at the forecast time and date.

Forecasting Point: Forecasts of forecasting points
- Forecasting point 1 / Overnight minimum (OM): minimum national demand 
    between half hour ending 00:30 and 07:30. 
- Forecasting point 2 / Daytime peak (DM): maximum national demand 
    between half hour ending 08:00 and 13:00. 
- Forecasting point 3 / Daytime minimum (Dm): minimum national demand 
    between half hour ending 13:30 and 16:30. 
- Forecasting point 4 / Evening peak (EM): maximum national demand 
    between half hour ending 17:00 and 24:00. 

Forecast Timestamp: The date and time at which the forecast was made.

In [ ]:
url = "https://api.neso.energy/api/3/action/datastore_search_sql"
resource_id = "9847e7bb-986e-49be-8138-717b25933fbb"

query = f"""
SELECT *
FROM "{resource_id}"
WHERE "TARGETDATE" >= '{start_date}' AND "TARGETDATE" <= '{end_date}'
ORDER BY "TARGETDATE" ASC
"""

params = {"sql": query}

r = requests.get(url, params=params)
if r.status_code == 200:
    demand_forecast = r.json()
else:
    raise Exception("Request Failed")

In [ ]:
# Unpack data into a dataframe
df_demand_dayahead = pd.DataFrame(demand_forecast['result']['records'])
df_demand_dayahead.columns

In [ ]:
df_demand_dayahead.drop(columns=['_full_text', '_id'], inplace=True)

Convert time data to a more usable format:

In [ ]:
df = df_demand_dayahead.copy()

# time strings padded with zeros to have standard format
cp_st_time_str = df['CP_ST_TIME'].astype(str).str.zfill(4)
cp_end_time_str = df['CP_END_TIME'].astype(str).str.zfill(4)

# identify rows with 2400 (these need to be converted to 0000)
cp_st_mask = df['CP_ST_TIME'] == '2400'
cp_end_mask = df['CP_END_TIME'] == '2400'

# replace 2400 with 0000
cp_st_time_str = cp_st_time_str.replace('2400', '0000')
cp_end_time_str = cp_end_time_str.replace('2400', '0000')

# combine date and time
df['CP_START'] = pd.to_datetime(df['TARGETDATE'].astype(str) + ' ' + cp_st_time_str, format='%Y-%m-%d %H%M')
df['CP_END'] = pd.to_datetime(df['TARGETDATE'].astype(str) + ' ' + cp_end_time_str, format='%Y-%m-%d %H%M')

# localise to Europe/London — handle DST transitions
df['CP_START'] = df['CP_START'].dt.tz_localize('Europe/London', ambiguous='NaT', nonexistent='NaT')
df['CP_END'] = df['CP_END'].dt.tz_localize('Europe/London', ambiguous='NaT', nonexistent='NaT')

# drop rows where timestamps couldn’t be localized (spring-forward gap)
# df = df.dropna(subset=['CP_START', 'CP_END'])

# add one day where the original time was 2400
df.loc[cp_st_mask, 'CP_START'] += pd.Timedelta(days=1)
df.loc[cp_end_mask, 'CP_END'] += pd.Timedelta(days=1)

# drop unnecessary columns
df.drop(columns=['CP_ST_TIME', 'CP_END_TIME', 'TARGETDATE'], inplace=True)
if (df_demand_dayahead['DAYSAHEAD'] == 1).all():
    df.drop(columns=['DAYSAHEAD'], inplace=True)
else:
    raise Exception("Expected day-ahead forecast.")

df_demand_dayahead = df

df_demand_dayahead.head(10)

*Definitions:*

**Cardinal Point:** Electricity demand fluctuates during a day depending
on how much energy people, businesses and industries are using at that
moment in time. As this electricity demand goes up and down we get
characteristic peaks and troughs, with some of these peaks and troughs
appearing every single day at similar times. These we call cardinal
points and are the points during the day that we forecast demand for.

**Cardinal Point Type:** Fixed, Trough or Peak. Cardinal points (CPs) 
can either be fixed (occur at a fixed time), trough (minimum demands 
during a set period of the day) or peak(maximum demands during a set 
period of the day). These are represented throught the first letter of 
the point type (F,T or P)

**Cardinal Point Start Time:** The time when a particular cardinal point 
(CP) starts during the day. This is given relative to the timezone in 
effect in the UK at the forecast time and date.

**Cardinal Point End Time:** The time when a particular cardinal point 
(CP) ends during the day. This is given relative to the timezone in 
effect in the UK at the forecast time and date.

**Forecasting Point:** Forecasts of forecasting points
- Forecasting point 1 / Overnight minimum (OM): minimum national demand 
    between half hour ending 00:30 and 07:30. 
- Forecasting point 2 / Daytime peak (DM): maximum national demand 
    between half hour ending 08:00 and 13:00. 
- Forecasting point 3 / Daytime minimum (Dm): minimum national demand
    between half hour ending 13:30 and 16:30. 
- Forecasting point 4 / Evening peak (EM): maximum national demand 
    between half hour ending 17:00 and 24:00. 

**Forecast Timestamp:** The date and time at which the forecast was made.

Move times to a uniformly spaced grid:

In [ ]:
df = df_demand_dayahead.copy()

# Assume cardinal point is at the midpoint of the start and end times
df['CP_MIDPOINT'] = df['CP_START'] + (df['CP_END'] - df['CP_START']) / 2

# Drop unnecessary columns
df.drop(columns=['CP_TYPE', 'F_Point', 'CARDINALPOINT', 'CP_START', 'CP_END'], inplace=True)

# Create a uniformly spaced 30-minute index in Europe/London time
df.set_index('CP_MIDPOINT', inplace=True)
start = df.index.min().floor('30min')
end = df.index.max().ceil('30min')
uniform_index = pd.date_range(start=start, end=end, freq='30min', tz='Europe/London')

# drop duplicate indices (happens when forecast gets updated)
df = df.loc[~df.index.duplicated(keep='first'), :]  # keep first forecast

# reindex the data to get uniform spacing
df = df.reindex(uniform_index, fill_value=np.nan)

# interpolate
df['FORECASTDEMAND'] = df['FORECASTDEMAND'].interpolate(method='linear')

# Add settlement date and settlement period
# Settlement periods are 1–48 per day, each 30 minutes long
df['SETTLEMENT_DATE'] = df.index.tz_convert('Europe/London').date
df = df.sort_index()
df['SETTLEMENT_PERIOD'] = df.groupby('SETTLEMENT_DATE').cumcount() + 1

# Reset index
df = df.reset_index().rename(columns={'index': 'DATETIME'})

# Drop unnecessary columns
df.drop(columns=['DATETIME', 'FORECAST_TIMESTAMP'], inplace=True)

# rename columns
df.rename(columns={'FORECASTDEMAND': 'DEMAND_FORECAST'}, inplace=True)

df_demand_dayahead = df.copy()

# display
df_demand_dayahead.head()

In [ ]:
df_demand_dayahead.tail()

In [ ]:
# HELPER FUNCTIONS:

import datetime

def keep_dates(
        df: pd.DataFrame, 
        date_range = (datetime.date(2021, 1, 1), datetime.date(2025, 1, 1)), 
        date_column: str = 'SETTLEMENT_DATE'):
    dates = df[date_column]
    df = df[(date_range[0] <= dates) & (dates < date_range[1])]
    df = df.reset_index(drop=True)
    return df

def last_sunday_of_month(year, month):
    """Find the last Sunday of a given month"""
    last_day = pd.Timestamp(year=year, month=month, day=1) + pd.offsets.MonthEnd(1)
    days_back = (last_day.dayofweek - 6) % 7  # Sunday is 6
    last_sunday = last_day - pd.Timedelta(days=days_back)
    return last_sunday.date()

def check_day_lengths(df: pd.DataFrame, date_column='SETTLEMENT_DATE'):
    """Check days have expected number of settlement periods."""
    daily_counts = df.groupby(df[date_column]).size()  # count settlement periods per day
    irregular_days = daily_counts[(daily_counts != 48) & (daily_counts != 46) & (daily_counts != 50)]
    assert len(irregular_days) == 0, "Found days with an irregular number of settlement periods."
    return True

def check_spring_dst(df: pd.DataFrame, date_column='SETTLEMENT_DATE'):
    daily_counts = df.groupby(df[date_column]).size()  # count settlement periods per day
    short_days = daily_counts[daily_counts == 46].index
    short_days = sorted(short_days)
    years_in_data = df[date_column].dt.year.unique()
    years = sorted(years_in_data)
    assert len(years) == len(short_days), "Expected only one short day per year."
    for (i, year) in enumerate(years):
        expected_date = last_sunday_of_month(year, 3)
        short_day = short_days[i].date()
        error_msg = f"Incorrect short day date in year {year}.\nExpected {expected_date}, got {short_day}."
        assert short_day == expected_date, error_msg
    return True

def check_autumn_dst(df: pd.DataFrame, date_column='SETTLEMENT_DATE'):
    daily_counts = df.groupby(df[date_column]).size()  # count settlement periods per day
    long_days = daily_counts[daily_counts == 50].index
    long_days = sorted(long_days)
    years_in_data = df[date_column].dt.year.unique()
    years = sorted(years_in_data)
    assert len(years) == len(long_days), "Expected only one short day per year."
    for (i, year) in enumerate(years):
        expected_date = last_sunday_of_month(year, 10)
        long_day = long_days[i].date()
        error_msg = f"Incorrect short day date in year {year}.\nExpected {expected_date}, got {long_day}."
        assert long_day == expected_date, error_msg
    return True

In [ ]:
# drop rows outside 2021-2024 date range
date_range = (datetime.date(2021, 1, 2), datetime.date(2025, 1, 1))
df_demand_dayahead = keep_dates(df_demand_dayahead, date_range=date_range, date_column='SETTLEMENT_DATE')

# check that number of rows is as expected
df = df_demand_dayahead.copy()
df['SETTLEMENT_DATE'] = pd.to_datetime(df['SETTLEMENT_DATE'])
assert check_day_lengths(df)
assert check_spring_dst(df)
assert check_autumn_dst(df)

df_demand_dayahead.head()

## Wind Generation Forecasts

These are forecasts for metered wind generation, i.e. not including embedded systems.

In [ ]:
url = "https://api.neso.energy/api/3/action/datastore_search_sql"
resource_id = "7524ec65-f782-4258-aaf8-5b926c17b966"

query = f"""
SELECT *
FROM "{resource_id}"
WHERE "Datetime_GMT" >= '{start_date}' AND "Datetime_GMT" <= '{end_date}'
ORDER BY "Datetime_GMT" ASC
"""

params = {"sql": query}

r = requests.get(url, params=params)
if r.status_code == 200:
    wind_forecast = r.json()
else:
    raise Exception("Request Failed")

In [ ]:
df_wind_dayahead = pd.DataFrame(wind_forecast['result']['records'])
df_wind_dayahead.columns

In [ ]:
# drop unnecessary columns
df_wind_dayahead.drop(columns=['_full_text', '_id', 'Forecast_Timestamp', 'Datetime_GMT'], inplace=True)

# rename to match conventions of other dataframes
df_wind_dayahead.rename(
    columns={
        'Incentive_forecast': 'WIND_FORECAST',
        'Settlement_period': 'SETTLEMENT_PERIOD',
        'Date': 'SETTLEMENT_DATE',
        'Capacity': 'WIND_CAPACITY'
        }, 
    inplace=True
    )

# convert strings to datetime
df_wind_dayahead['SETTLEMENT_DATE'] = pd.to_datetime(df_wind_dayahead['SETTLEMENT_DATE']).dt.date

df_wind_dayahead.head()

In [ ]:
df_wind_dayahead.tail()

In [ ]:
# drop rows outside 2021-2024 date range
date_range = (datetime.date(2021, 1, 2), datetime.date(2025, 1, 1))
df_wind_dayahead = keep_dates(df_wind_dayahead, date_range=date_range, date_column='SETTLEMENT_DATE')

# check that number of rows is as expected
df = df_wind_dayahead.copy()
df['SETTLEMENT_DATE'] = pd.to_datetime(df['SETTLEMENT_DATE'])
assert check_day_lengths(df)
assert check_spring_dst(df)
assert check_autumn_dst(df)

df_wind_dayahead.head()

## Embedded Wind and Solar Forecasts

Load from CSVs available on NESO website since the files are large which makes API calls very slow:

In [ ]:
import os

base_dir = os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "neso")
years = [2021, 2022, 2023, 2024]
dfs = []

for year in years:
    load_path = os.path.join(base_dir, f"embedded_archive_{year}.csv")
    df = pd.read_csv(load_path)  # file size is 300+ MB

    # combine separate date and time columns + convert other timestamp columns to correct format
    dates_str = (pd.to_datetime(df['DATE_GMT']).dt.date).astype("string")
    df['DATETIME_GMT'] = pd.to_datetime(dates_str + 'T' + df['TIME_GMT'], format='ISO8601', utc=True)
    df.drop(columns=['DATE_GMT', 'TIME_GMT'], inplace=True)
    df['Forecast_Datetime'] = pd.to_datetime(df['Forecast_Datetime'])
    df['SETTLEMENT_DATE'] = pd.to_datetime(df['SETTLEMENT_DATE']).dt.date

    # keep last forecast made on the previous day before 09:00 (market close is at 10:20)
    forecast_date = df['Forecast_Datetime'].dt.date
    target_date = df['SETTLEMENT_DATE']
    forecast_hour = df['Forecast_Datetime'].dt.hour
    is_previous_day = (target_date - forecast_date) == pd.Timedelta(days=1)
    is_morning_forecast = (forecast_hour >= 0) & (forecast_hour < 9)
    df = df[is_previous_day & is_morning_forecast]
    
    # remove duplicates, keeping the last forecast made before 09:00 for each DATETIME_GMT
    df = df.sort_values('Forecast_Datetime')
    df = df.drop_duplicates(subset='DATETIME_GMT', keep='last')
    df.reset_index(drop=True, inplace=True)

    # drop unnecessary rows to reduce memory requirements
    df.drop(columns=['Forecast_Datetime', 'DATETIME_GMT'], inplace=True)

    dfs.append(df)

df_emb_dayahead = pd.concat(dfs, ignore_index=True)

print(df_emb_dayahead.info())

df_emb_dayahead.head()

In [ ]:
# drop rows outside 2021-2024 date range
date_range = (datetime.date(2021, 1, 2), datetime.date(2025, 1, 1))
df_emb_dayahead = keep_dates(df_emb_dayahead, date_range=date_range, date_column='SETTLEMENT_DATE')

# check that number of rows is as expected
df = df_emb_dayahead.copy()
df['SETTLEMENT_DATE'] = pd.to_datetime(df['SETTLEMENT_DATE'])
assert check_day_lengths(df)
assert check_spring_dst(df)
assert check_autumn_dst(df)

df_emb_dayahead.head()

## Merge forecast data

Merge forecasts for (a) demand (b) metered wind generation, and (c) embedded wind and solar generation to form one convenient dataframe.

In [ ]:
df_merged = pd.merge(df_demand_dayahead, df_emb_dayahead, how='inner', on=['SETTLEMENT_DATE', 'SETTLEMENT_PERIOD'])
df_merged = pd.merge(df_merged, df_wind_dayahead, how='inner', on=['SETTLEMENT_DATE', 'SETTLEMENT_PERIOD'])
df_merged.head()

In [ ]:
# drop rows outside 2021-2024 date range
date_range = (datetime.date(2021, 1, 2), datetime.date(2025, 1, 1))
df_merged = keep_dates(df_merged, date_range=date_range, date_column='SETTLEMENT_DATE')

# check that number of rows is as expected
df = df_merged.copy()
df['SETTLEMENT_DATE'] = pd.to_datetime(df['SETTLEMENT_DATE'])
assert check_day_lengths(df)
assert check_spring_dst(df)
assert check_autumn_dst(df)

df_merged.head()

In [ ]:
# # Write to disk
# save_path = os.path.join(os.path.dirname(os.getcwd()), "data", "processed", "forecast_data.csv")
# df_merged.to_csv(save_path, index=False)